# IFT3700 A2023 - TP

# Introduction

Dans le cadre de notre travail final pour le cours IFT3700 A2023, notre équipe composée de [TODO: Noms des Membres] a entrepris la tâche de collecter, nettoyer et analyser des données numériques et catégoriques relatives à divers pays du monde. Ces données ont été extraites de 40 tableaux provenant de sources fiables sur Wikipédia (disponible dans l'énoncé du TP). Notre objectif était de nettoyer ces données, de les discrétiser en format binaire et de mener une analyse approfondie pour en extraire des insights pertinents.

Notez que certains imports sont nécessaires

    Pour une meilleure visualisation de notre travail, retrouvez notre repository au lien suivant:
https://github.com/mildshield14/ift3700_tp

# Méthodologie

## Collecte des données

Nous avons collecté les données à partir des sources spécifiées dans l'énoncé. Nous avons utilisé des scripts afin de mener à bien le webscraping.

Notre script est disponible ci-dessous : 

In [ ]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import unquote
from urllib.parse import unquote, urlsplit
import re

# Read URLs from the PHP file
with open("view.php", 'r') as file:
    php_code = file.read()

# Extract URLs using regex
urls = re.findall(r'"ulnk_url":"(https://en\.wikipedia\.org/wiki/[^"]+)"', php_code)

# Initialize a counter for file names
count = 1

# Iterate through the URLs
for url in urls:
    response = requests.get(url)
    time.sleep(1)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all tables with the specified class
    tables = soup.find_all('table', {'class': 'wikitable'})

     # Get page title from the URL
    page_title = unquote(urlsplit(url).path.split('/')[-1])

    # Initialize an empty list to store DataFrames for each table on the current page
    dfs_per_page = []

    # Iterate through the list of tables and convert each to a DataFrame
    for i, table in enumerate(tables):
        df = pd.read_html(str(table), header=0)[0]  # Assuming headers are in the first row
        dfs_per_page.append(df)


        # Create a folder for each page if it doesn't exist in another folder (for duplicated columns)
        page_folder = f'csv_duplicated_columns/{count}.{page_title}'
        os.makedirs(page_folder, exist_ok=True)
        
        # Create a folder for each page if it doesn't exist
        page_folder = f'csv_rawdata/{count}.{page_title}'
        os.makedirs(page_folder, exist_ok=True)

        # Write each DataFrame to a separate CSV file
        filename = f'{page_folder}/{count}.{i + 1} table.csv'
        with open(filename, 'w', newline='') as file:
            df.to_csv(file, index=False)


    # Increment the counter
    count += 1

print("done")


## Nettoyage des Données

Nous avons commencé par extraire les colonnes souhaitées pour chaque fichier à l'aide de scripts. Ainsi, nous nous sommes retrouvés avec ce que l'on appelle les "raw data".

Cela a déclenché la phase complexe et longue du nettoyage des données.

De ce fait, chacun des memebres de l'équipe avait pour but de nettoyer les données de 10 fichiers.

Voici un exemple de sacript utilisé pour avoir un fichier à 2 colonnes; pays-carcteristique


In [ ]:
import pandas as pd

# Load the data into a DataFrame
df = pd.read_csv("/Users/vennilasooben/Downloads/ift3700_tp/csv_duplicated_columns/9.Importance_of_religion_by_country/9.1 cleaned.csv")
print(df.columns)
# Select only the desired columns
selected_columns = ['Country', 'Importance of Religion(9)']
df = df[selected_columns]

# Clean the "Country" column
df['Country'] = df['Country'].str.replace(r'[^A-Za-z0-9\s]+', '', regex=True)

df['Importance of Religion(9)'] = df['Importance of Religion(9)'].str.replace('%', '')
df['Importance of Religion(9)'] = pd.to_numeric(df['Importance of Religion(9)'], errors='coerce')

# Save the cleaned data to a new CSV file
df.to_csv('9.1 cleaned.csv', index=False)

Nous avons rencontré plusieurs problèmes que nous énumérerons, tout en proposant également les solutions que nous avons appliquées.

##### 1) Noms différents dans chaque fichier

Pour résoudre ce problème, notre première idée fut d'utiliser les concepts de distances et de similarités vus en cours afin de trouver les pays que nous souhaitons qui seraient prédéfinis. 

Cependant, entre DR, Democratic Republic of Congo, Congo, Republic of Congo, etc., et le conseil du professeur en classe de ne pas abuser des scripts, nous avons pris la décision de le faire manuellement. 

Ainsi, nous avons utiliser deux scripts qui permettront de mieux visualiser et faire uniquement des changements que l'on veut.

- Le premier script permet d'obtenir une aide visuelle des pays en ordre alphabétique. Cela permet de voir des différences telles que "United States" et "United States of America".

- Ensuite, nous avons utilisé un deuxième script qui parcourt le dossier de fichiers CSV et qui cherche la chaîne de caractères à remplacer et la remplace. En effet nous mettons en entrée "United States" et "United States of America" et le remplacement se fait de manière automatique.

Vous trouverez ci-dessous les deux scripts utilisés:

In [ ]:
# Script permettant d'obtenir une aide visuelle des noms de chaque pays.
import pandas as pd

# Load the CSV file into a DataFrame
csv_file_path = '/Users/vennilasooben/Downloads/ift3700_tp/cleaning_whole_data/final_table.csv'
df = pd.read_csv(csv_file_path)

# Extract the first column with names
names_column = df.iloc[:, 0]

# Create a dictionary to store data based on the first letter
data_by_first_letter = {}
for name in names_column:
    first_letter = name[0].upper()
    if first_letter not in data_by_first_letter:
        data_by_first_letter[first_letter] = []
    data_by_first_letter[first_letter].append(name)

# Create a new DataFrame based on the dictionary
new_df = pd.DataFrame(data_by_first_letter.items(), columns=['First Letter', 'Names'])

# Save the new DataFrame to a CSV file
new_csv_file_path = 'organized_data_country_names.csv'
new_df.to_csv(new_csv_file_path, index=False)

print(f"Data has been organized and saved to {new_csv_file_path}.")


In [111]:
# Script permettant de remplacer les noms des pays
import os
import pandas as pd

def process_folder(folder_path, replace_this, replace_with):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".csv"):
                file_path = os.path.join(root, file)
                df = pd.read_csv(file_path)

                if "Country" in df.columns:
                    # Replace the specified string in the "Country" column
                    df["Country"] = df["Country"].str.replace(replace_this, replace_with)

                    # Save the modified DataFrame back to the same file
                    df.to_csv(file_path, index=False)

def main():
    folder_path = "/Users/vennilasooben/Downloads/ift3700_tp/csv_duplicated_columns"
    replace_this = input("replacw what?")
    replace_with = input("replace with what?")
    
    process_folder(folder_path, replace_this, replace_with)

if __name__ == "__main__":
    main()

Voici des exemples de quelques conventions (dont les changements revenaient souvent) que nous avons trouvés pertinents afin de faciliter tous nos parsings.

- United States
- Democratic Republic of the Congo
- Republic of the Congo
- Myanmar
- Macau
- Ivory Coast
- Guinea-Bissau
- Sao Tome and Principe
- British Virgin Islands
- US Virgin Islands (par défaut si non mentionné; décision suite à une vérification de drapeau)
- suppression de tous les accents
- et plus encore

##### 2) Danc certains cas, nous nous sommes retrouvés avec des valeurs en double voire triple.

   - Ce à quoi on s'attendait:
   
   United Kingdom, England and Wales ainsi que United Kingdom, Great Britain and Northern Islands y étaient. Dans ce cas, nous avons pris la décision de faire la moyenne car la différence entre les 2 était de 0.2. Ainsi nous avons choisi 1.1.


   - L'inattendu:
   
   Cependant, nous avons également fait face à des surprises dû à un manque de connaissance sur le sujet. Dans ce cas, des recherches ont été nécéssaires. 
   
   Par exemple, Iraq et Iraq Kurdistan ne sont pas les mêmes. Bien que nous ayons supprimés Iraq Kurdistan dû à un manque important de données non-négligeables, pour cette étape du nettoyage, cette information a son importance car la même chose aurait pu arriver avec des pays différents si nos connaissances en géographie était vraiment limité..

    (TODO: check if removed iraq kurdistan)


Avant de passer à l'étape gestion des valeurs manquantes, nous avons combiné les données. Ceci permet non seulemnt une meilleure visualisation mais aussi facilite la gestion car certains fichiers ont des pays que dautres nont pas, necessitanr ainsi eux aussi des donnees de remplacement.

Voici de script qui merge le tout:

In [ ]:
import os
import pandas as pd
import re

def process_folder(folder_path, output_file):
    all_data = pd.DataFrame(columns=["Country"])

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if "cleaned.csv" in file:
                file_path = os.path.join(root, file)
                df = pd.read_csv(file_path)

                if "Country" in df.columns:
                    # Extract the "Country" and the second column data
                    country_column = df["Country"]
                    second_column_label = df.columns[1]  
                    second_column_data = df.iloc[:, 1]

                    # Create a DataFrame with "Country" and the second column data
                    result_df = pd.DataFrame({"Country": country_column, f"{second_column_label}": second_column_data})

                    # Merge with the existing data using "Country" as the key
                    all_data = pd.merge(all_data, result_df, on="Country", how="outer")

    all_data.to_csv(output_file, index=False)

# Function to extract the number from the column names
def extract_number_from_column(column_name):
        try:
            #asked chatgpt about regex to extract (number) from second columns name and got that answer
          numbers = re.findall(r'\((\d+)\)', column_name)
          return tuple(map(int, numbers))
        except ValueError:
         print("put (number) in youe column header for example: Intentional Homicide Rate(4)")
         return float('inf')   # If no number is found, place it at the end for now temporarily

if __name__ == "__main__":

    folder_path = "/Users/vennilasooben/Downloads/ift3700_tp/csv_duplicated_columns"
    output_file = "/Users/vennilasooben/Downloads/ift3700_tp/cleaning_whole_data/final_table.csv"
    process_folder(folder_path, output_file)

    # Read the CSV file into a DataFrame
    df = pd.read_csv(output_file)

    # Sort the columns based on the numbers in parentheses
    sorted_columns =  sorted(df.columns[1:], key=extract_number_from_column)

    # Reorganize the DataFrame columns
    df = df[["Country"] + sorted_columns]

    # Save the resulting DataFrame to a CSV file
    df.to_csv(output_file, index=False)



### Gestion des valeurs manquantes

In [112]:
#TODO

### Gestion des outliers

Nous avons ...

In [113]:
#TODO 

### Discrétisation des Caractéristiques


Nous avons ....

In [114]:
#TODO

## Analyse des Données

### Calcul des statistiques descriptives

Pour chaque ....

In [115]:
#TODO

### Calcul des Corrélations

Calcul coeff de corr etc

In [116]:
#TODO

### Prédictions

Ref lin etc

In [117]:
#TODO

### Visualisation et Représentations

Nous avons ...

In [118]:
#TODO

# Résultats

## Statistiques descriptives

[TODO : Inclure mean median etc]

## Corrélations

[TODO]

## Prédictions

[TODO : Regr Lin, class bayes etc ]

## Visualisation

[TODO- Inclure images]

# Conclusion

Dans ce trvail, ...